In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import joblib

In [2]:
# path configuration

base_path = "C:\kuliah\Semester 5\Machine learning\project 3 matkul\Mangosteen.v1i.multiclass"

#train_path = os.path.join(base_path, "train")
#image_train = train_path
#label_train = os.path.join(train_path, "train_classes.csv")
#
#test_path = os.path.join(base_path, "test")
#image_test = test_path
#label_test = os.path.join(test_path, "test_classes.csv")

manggis_path = os.path.join(base_path, "all_dataset")
label_path = os.path.join(manggis_path, "manggis_label.csv")
image_path = manggis_path

In [3]:
# load data function

# load csv + one hot -> label tunggal
def load_label(label_path):
    df = pd.read_csv(os.path.join(manggis_path, "manggis_label.csv"))
    label_columns = [" mangosteen 1", " mangosteen 2", " mangosteen 3", " mangosteen 4", " mangosteen 5-6"]
    
    df[label] = df[label_colums].idxmax(axis=1).str.strip()
    
    return df

# load gambar dari all_dataset
def load_dataset(df, img_dir):
    X = []
    y = []
        
    for idx, row in df.iterrows():
        filename = row["filename"]
        label = row["label"]
        
        
        img_path = os.path.join(img_dir, filename)
        if not os.path.exists(img_path):
            print("file tidak ditemukan", img_path)
            continue
        
        img = cv2.imread(img_path)
        if img is None:
            print("gambar gagal dibaca", img_path)
            continue
        
        img = cv2.resize(img, (64, 64))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        X.append(img)
        y.append(label)
        
    return np.array(data), np.array(labels)

In [4]:
# Train dan validate dengan K-Fold
def train_kfold(X, y, n_splits=5):
    kf =KFold(n_splits=n_splits, shuffle=True)

    fold = 1
    accuracies = []

    for train_index, test_index in kf.split(X, y):
        X_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = DecisionTreeClassifier(max_depth= 50)
        model.fit(X_train, y_train)

        # evaluate
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)


        print(f"Fold {fold} Accuracy: {acc}")
        fold += 1
    
    print("\n rata-rata Akurasi", np.mean(accuracies))
    return np.mean(accuracies)

In [ ]:
def main():
    df = load_label(label_path)
    
    X, y = load_dataset(df, image_path)
    print("Total gambar terbaca:", len(X))
    
    model, avg_acc = train_kfold(X, y, n_splits=5)
    
    #save model
    save_path = os.path.join(base_path, "decision_tree_manggis.pkl")
    joblib.dump(model, save_path)
    
if __name__ == "__main__":
    main()
    